In [1]:
!pip install torch_dct

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import os
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
from matplotlib import pyplot as plt
from torch import nn
from torchvision.transforms import transforms
from models.models import Simple_CNN




class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targeted intermediate layers """
    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(
                    self.save_activation))
            # Backward compatibility with older pytorch versions:
            if hasattr(target_layer, 'register_full_backward_hook'):
                self.handles.append(
                    target_layer.register_full_backward_hook(
                        self.save_gradient))
            else:
                self.handles.append(
                    target_layer.register_backward_hook(
                        self.save_gradient))
    def save_activation(self, module, input, output):
        activation = output
        if self.reshape_transform is not None:
            activation = self.reshape_transform(activation)
        self.activations.append(activation.cpu().detach())
    def save_gradient(self, module, grad_input, grad_output):
        # Gradients are computed in reverse order
        grad = grad_output[0]
        if self.reshape_transform is not None:
            grad = self.reshape_transform(grad)
        self.gradients = [grad.cpu().detach()] + self.gradients
    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)
    def release(self):
        for handle in self.handles:
            handle.remove()
class GradCAM:
    def __init__(self,
                 model,
                 target_layers,
                 reshape_transform=None,
                 use_cuda=False):
        self.model = model.eval()
        self.target_layers = target_layers
        self.reshape_transform = reshape_transform
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.activations_and_grads = ActivationsAndGradients(
            self.model, target_layers, reshape_transform)
    """ Get a vector of weights for every channel in the target layer.
        Methods that return weights channels,
        will typically need to only implement this function. """
    @staticmethod
    def get_cam_weights(grads):
        return np.mean(grads, axis=(2, 3), keepdims=True)
    @staticmethod
    def get_loss(output, target_category):
        loss = 0
        for i in range(len(target_category)):
            loss = loss + output[i, target_category[i]]
        return loss
    def get_cam_image(self, activations, grads):
        weights = self.get_cam_weights(grads)
        weighted_activations = weights * activations
        cam = weighted_activations.sum(axis=1)
        return cam
    @staticmethod
    def get_target_width_height(input_tensor):
        width, height = input_tensor.size(-1), input_tensor.size(-2)
        return width, height
    def compute_cam_per_layer(self, input_tensor):
        activations_list = [a.cpu().data.numpy()
                            for a in self.activations_and_grads.activations]
        grads_list = [g.cpu().data.numpy()
                      for g in self.activations_and_grads.gradients]
        target_size = self.get_target_width_height(input_tensor)
        cam_per_target_layer = []
        # Loop over the saliency image from every layer
        for layer_activations, layer_grads in zip(activations_list, grads_list):
            cam = self.get_cam_image(layer_activations, layer_grads)
            cam[cam < 0] = 0  # works like mute the min-max scale in the function of scale_cam_image
            scaled = self.scale_cam_image(cam, target_size)
            cam_per_target_layer.append(scaled[:, None, :])
        return cam_per_target_layer
    def aggregate_multi_layers(self, cam_per_target_layer):
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        cam_per_target_layer = np.maximum(cam_per_target_layer, 0)
        result = np.mean(cam_per_target_layer, axis=1)
        return self.scale_cam_image(result)
    @staticmethod
    def scale_cam_image(cam, target_size=None):
        result = []
        for img in cam:
            img = img - np.min(img)
            img = img / (1e-7 + np.max(img))
            if target_size is not None:
                img = cv2.resize(img, target_size)
            result.append(img)
        result = np.float32(result)
        return result
    def __call__(self, input_tensor, target_category=None):
        if self.cuda:
            input_tensor = input_tensor.cuda()
        # 正向传播得到网络输出logits(未经过softmax)
        output = self.activations_and_grads(input_tensor)
        if isinstance(target_category, int):
            target_category = [target_category] * input_tensor.size(0)
        if target_category is None:
            target_category = np.argmax(output.cpu().data.numpy(), axis=-1)
            print(f"category id: {target_category}")
        else:
            assert (len(target_category) == input_tensor.size(0))
        self.model.zero_grad()
        loss = self.get_loss(output, target_category)
        loss.backward(retain_graph=True)
        # In most of the saliency attribution papers, the saliency is
        # computed with a single target layer.
        # Commonly it is the last convolutional layer.
        # Here we support passing a list with multiple target layers.
        # It will compute the saliency image for every image,
        # and then aggregate them (with a default mean aggregation).
        # This gives you more flexibility in case you just want to
        # use all conv layers for example, all Batchnorm layers,
        # or something else.
        cam_per_layer = self.compute_cam_per_layer(input_tensor)
        return self.aggregate_multi_layers(cam_per_layer)
    def __del__(self):
        self.activations_and_grads.release()
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True
def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255
    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")
    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)
def center_crop_img(img: np.ndarray, size: int):
    h, w, c = img.shape
    if w == h == size:
        return img
    if w < h:
        ratio = size / w
        new_w = size
        new_h = int(h * ratio)
    else:
        ratio = size / h
        new_h = size
        new_w = int(w * ratio)
    img = cv2.resize(img, dsize=(new_w, new_h))
    if new_w == size:
        h = (new_h - size) // 2
        img = img[h: h+size]
    else:
        w = (new_w - size) // 2
        img = img[:, w: w+size]
    return img

In [2]:
def main(img_path):
  
    net = Simple_CNN(class_num=11)
    device = torch.device("cpu")
    net.load_state_dict(torch.load("/mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_24_test91.75_acc_AUC_80.8_OSCR_77.09646012396702.pth",
                                   map_location=device)['state_dict'])  # 载入训练的resnet模型权重，你将训练的模型权重放到当前文件夹下即可
    target_layers = [net.lateral_convs[-1]] #这里是 看你是想看那一层的输出，我这里是打印的resnet最后一层的输出，你也可以根据需要修改成自己的
    print(target_layers)
    data_transform = transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    # 导入图片
    # img_path = "/mnt/workspace/POSE/dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001529.png"#这里是导入你需要测试图片
    image_size = 128#训练图像的尺寸，在你训练图像的时候图像尺寸是多少这里就填多少
    assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    img = Image.open(img_path).convert('RGB')#将图片转成RGB格式的
    img = np.array(img, dtype=np.uint8) #转成np格式
    img_original = img.copy()  # 保存原始图像副本用于显示
    img = center_crop_img(img, image_size) #将测试图像裁剪成跟训练图片尺寸相同大小的
    # [C, H, W]
    img_tensor = data_transform(img)#简单预处理将图片转化为张量
    # expand batch dimension
    # [C, H, W] -> [N, C, H, W]
    input_tensor = torch.unsqueeze(img_tensor, dim=0) #增加一个batch维度
    cam = GradCAM(model=net, target_layers=target_layers, use_cuda=False)
    grayscale_cam = cam(input_tensor=input_tensor)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(img.astype(dtype=np.float32) / 255.,
                                      grayscale_cam,
                                      use_rgb=True)
    # plt.imshow(visualization)
    # plt.show()
    print(img_path)
    # 使用matplotlib的子图功能同时显示原图和热力图
    plt.figure(figsize=(12, 6))
    
    # 显示原图
    plt.subplot(1, 2, 1)
    plt.imshow(img_original)
    plt.title('Original Image')
    plt.axis('off')
    
    # 显示热力图
    plt.subplot(1, 2, 2)
    plt.imshow(visualization)
    plt.title('GradCAM Visualization')
    plt.axis('off')
    
    plt.tight_layout()  # 调整布局
    plt.show()
    
# if __name__ == '__main__':
#     main()

In [ ]:
def parse_image_paths(a):
    """解析字符串a，提取图像路径"""
    paths = []
    for line in a.split('\n'):
        if line.strip():
            # 分割路径和标签，取路径部分
            path = line.split('\t')[0]
            # 转换路径格式：将'./'替换为'/mnt/workspace/POSE/'
            path = path.replace('./', '/mnt/workspace/POSE/')
            paths.append(path)
    return paths


paths = '''./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000568.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001529.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017180.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017646.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018748.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007439.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015459.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013337.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004199.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001003.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013611.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018550.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000128.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006682.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016463.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007422.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014800.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016269.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007787.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007519.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014098.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015986.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014522.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007516.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008154.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009354.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010670.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017662.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008807.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000195.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000936.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019707.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008441.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006363.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018026.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018065.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005105.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012143.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005278.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017591.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002143.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004320.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013409.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016817.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018615.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001965.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005638.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012888.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000856.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015049.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018147.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017277.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010971.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017786.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003565.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003528.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012879.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004084.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016062.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013515.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011190.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001637.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001446.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016202.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005223.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006853.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017095.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002782.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004704.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014120.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004393.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019176.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013111.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018117.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009655.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002462.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003061.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004845.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008684.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014486.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013081.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007914.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010174.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008056.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019123.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006373.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014992.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018854.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019686.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013407.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001941.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007625.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009670.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015791.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018934.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017631.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014392.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014887.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013593.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007685.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000763.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003747.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014472.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001008.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008029.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008020.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004330.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014152.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003768.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006869.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001854.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007859.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008123.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006581.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006026.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002826.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018740.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008340.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013907.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005862.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003782.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013096.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013104.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006102.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009495.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001493.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017749.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013390.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001130.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004829.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015322.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010421.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000372.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005754.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012065.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010790.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013740.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002957.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001706.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019961.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017056.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001959.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002170.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015095.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007522.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013020.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016907.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005298.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013599.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019916.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006568.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007585.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000978.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006017.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010248.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001281.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013273.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008009.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004628.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013636.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006366.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009601.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011186.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000559.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005430.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007207.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008480.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001676.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003255.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006488.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006821.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012448.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003094.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013542.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002266.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007209.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006978.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004286.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019396.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011798.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013305.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008585.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019659.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009982.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001415.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006441.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002992.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006101.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003678.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004239.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006949.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019254.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004318.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006088.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018177.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013842.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010261.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004357.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017229.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003373.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017477.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017019.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013479.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004912.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017298.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000447.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008232.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013016.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001876.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001638.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004136.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008342.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000671.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006234.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001498.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018938.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001825.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017108.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008663.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007537.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015681.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012461.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010805.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015286.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018465.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004025.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005151.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007313.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017434.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009393.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006893.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006374.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009279.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008112.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014518.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008239.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006152.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000572.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006973.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000171.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003598.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006672.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002127.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013322.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005130.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011858.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010624.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002259.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017959.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011006.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012427.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011049.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015385.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008905.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019899.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014456.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004386.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017031.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003913.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016685.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007609.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011982.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011029.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001432.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013165.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003577.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012887.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005053.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019032.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017884.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017078.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014243.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018673.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001560.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019987.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013958.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011913.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016656.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005082.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002829.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001259.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005846.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003688.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007296.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019182.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000105.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007649.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005718.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002508.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002749.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015295.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002023.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009077.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014918.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005072.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005478.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002789.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015612.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000038.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000413.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010167.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002726.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017208.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015771.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005475.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013661.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007597.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005240.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004247.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010336.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016776.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003960.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003886.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017886.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008507.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010234.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012731.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003986.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017727.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010842.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018448.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008423.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010739.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010866.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000850.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015342.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000951.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006877.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001338.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012660.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010512.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003611.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004989.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001867.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000745.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011771.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010195.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010694.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016875.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006408.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017595.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006089.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013880.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017310.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019197.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016160.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013223.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007793.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007601.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012797.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009734.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005248.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017552.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014610.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017915.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006452.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007224.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001963.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012170.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019381.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017411.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002022.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017664.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010240.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004404.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005585.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012241.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012361.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004197.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010897.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017846.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011542.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018779.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011754.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017956.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006781.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003358.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002007.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013457.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011241.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006847.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002627.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009016.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000818.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012136.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019883.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010994.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011475.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014285.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008808.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019122.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005896.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002831.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019477.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009500.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016994.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019117.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010349.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008399.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008717.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018355.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014393.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003594.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009732.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00014541.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001491.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004216.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015975.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005355.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000677.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006621.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019369.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006340.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008178.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010312.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012522.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007194.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011609.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017345.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005697.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006510.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003264.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00002596.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019599.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011569.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007206.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005281.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018338.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007057.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019681.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018080.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015683.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017482.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000092.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009932.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003235.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017074.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013424.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013925.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009072.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011000.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012865.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009005.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005199.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017292.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017625.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015142.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010480.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016477.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000243.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004435.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00011261.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013058.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008851.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00010959.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003761.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00019490.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008244.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009451.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017463.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005596.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004848.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003775.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016797.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005489.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00013976.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00000537.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003003.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017125.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00007333.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015665.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00012509.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009092.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016618.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005227.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00001984.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00009789.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008504.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015794.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00018543.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00005175.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016302.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s3_00015625.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s0_00003265.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016611.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006341.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004599.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006343.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00006331.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s1_00004332.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00017028.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s2_00008766.png	3
./dataset/CNN/GAN/Faces_HQ/ProGAN/train/ProGAN_celebhp_s4_00016497.png	3
./dataset/T2I/stable_diffusion_v21/train/000000313588.png	10
./dataset/T2I/stable_diffusion_v21/train/000000112626.png	10
./dataset/T2I/stable_diffusion_v21/train/000000419201.png	10
./dataset/T2I/stable_diffusion_v21/train/000000578489.png	10
./dataset/T2I/stable_diffusion_v21/train/000000456015.png	10
./dataset/T2I/stable_diffusion_v21/train/000000073946.png	10
./dataset/T2I/stable_diffusion_v21/train/000000447611.png	10
./dataset/T2I/stable_diffusion_v21/train/000000243204.png	10
./dataset/T2I/stable_diffusion_v21/train/000000439715.png	10
./dataset/T2I/stable_diffusion_v21/train/000000520531.png	10
./dataset/T2I/stable_diffusion_v21/train/000000248631.png	10
./dataset/T2I/stable_diffusion_v21/train/000000238410.png	10
./dataset/T2I/stable_diffusion_v21/train/000000419096.png	10
./dataset/T2I/stable_diffusion_v21/train/000000203639.png	10
./dataset/T2I/stable_diffusion_v21/train/000000560911.png	10
./dataset/T2I/stable_diffusion_v21/train/000000355325.png	10
./dataset/T2I/stable_diffusion_v21/train/000000387916.png	10
./dataset/T2I/stable_diffusion_v21/train/000000107087.png	10
./dataset/T2I/stable_diffusion_v21/train/000000368940.png	10
./dataset/T2I/stable_diffusion_v21/train/000000416104.png	10
./dataset/T2I/stable_diffusion_v21/train/000000061584.png	10
./dataset/T2I/stable_diffusion_v21/train/000000390301.png	10
./dataset/T2I/stable_diffusion_v21/train/000000242934.png	10
./dataset/T2I/stable_diffusion_v21/train/000000291791.png	10
./dataset/T2I/stable_diffusion_v21/train/000000260657.png	10
./dataset/T2I/stable_diffusion_v21/train/000000068093.png	10
./dataset/T2I/stable_diffusion_v21/train/000000408774.png	10
./dataset/T2I/stable_diffusion_v21/train/000000162366.png	10
./dataset/T2I/stable_diffusion_v21/train/000000146498.png	10
./dataset/T2I/stable_diffusion_v21/train/000000222118.png	10
./dataset/T2I/stable_diffusion_v21/train/000000147518.png	10
./dataset/T2I/stable_diffusion_v21/train/000000261712.png	10
./dataset/T2I/stable_diffusion_v21/train/000000318455.png	10
./dataset/T2I/stable_diffusion_v21/train/000000270474.png	10
./dataset/T2I/stable_diffusion_v21/train/000000410880.png	10
./dataset/T2I/stable_diffusion_v21/train/000000412531.png	10
./dataset/T2I/stable_diffusion_v21/train/000000450303.png	10
./dataset/T2I/stable_diffusion_v21/train/000000143961.png	10
./dataset/T2I/stable_diffusion_v21/train/000000521405.png	10
./dataset/T2I/stable_diffusion_v21/train/000000472030.png	10
./dataset/T2I/stable_diffusion_v21/train/000000249550.png	10
./dataset/T2I/stable_diffusion_v21/train/000000339823.png	10
./dataset/T2I/stable_diffusion_v21/train/000000393282.png	10
./dataset/T2I/stable_diffusion_v21/train/000000061418.png	10
./dataset/T2I/stable_diffusion_v21/train/000000525322.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426203.png	10
./dataset/T2I/stable_diffusion_v21/train/000000540466.png	10
./dataset/T2I/stable_diffusion_v21/train/000000021167.png	10
./dataset/T2I/stable_diffusion_v21/train/000000036936.png	10
./dataset/T2I/stable_diffusion_v21/train/000000290081.png	10
./dataset/T2I/stable_diffusion_v21/train/000000065455.png	10
./dataset/T2I/stable_diffusion_v21/train/000000327769.png	10
./dataset/T2I/stable_diffusion_v21/train/000000556000.png	10
./dataset/T2I/stable_diffusion_v21/train/000000407524.png	10
./dataset/T2I/stable_diffusion_v21/train/000000039951.png	10
./dataset/T2I/stable_diffusion_v21/train/000000519569.png	10
./dataset/T2I/stable_diffusion_v21/train/000000386134.png	10
./dataset/T2I/stable_diffusion_v21/train/000000112298.png	10
./dataset/T2I/stable_diffusion_v21/train/000000498857.png	10
./dataset/T2I/stable_diffusion_v21/train/000000010977.png	10
./dataset/T2I/stable_diffusion_v21/train/000000051326.png	10
./dataset/T2I/stable_diffusion_v21/train/000000130586.png	10
./dataset/T2I/stable_diffusion_v21/train/000000044877.png	10
./dataset/T2I/stable_diffusion_v21/train/000000413395.png	10
./dataset/T2I/stable_diffusion_v21/train/000000512648.png	10
./dataset/T2I/stable_diffusion_v21/train/000000568690.png	10
./dataset/T2I/stable_diffusion_v21/train/000000416745.png	10
./dataset/T2I/stable_diffusion_v21/train/000000545730.png	10
./dataset/T2I/stable_diffusion_v21/train/000000122672.png	10
./dataset/T2I/stable_diffusion_v21/train/000000049091.png	10
./dataset/T2I/stable_diffusion_v21/train/000000429761.png	10
./dataset/T2I/stable_diffusion_v21/train/000000135604.png	10
./dataset/T2I/stable_diffusion_v21/train/000000156372.png	10
./dataset/T2I/stable_diffusion_v21/train/000000062554.png	10
./dataset/T2I/stable_diffusion_v21/train/000000329827.png	10
./dataset/T2I/stable_diffusion_v21/train/000000505638.png	10
./dataset/T2I/stable_diffusion_v21/train/000000053529.png	10
./dataset/T2I/stable_diffusion_v21/train/000000253386.png	10
./dataset/T2I/stable_diffusion_v21/train/000000447200.png	10
./dataset/T2I/stable_diffusion_v21/train/000000076261.png	10
./dataset/T2I/stable_diffusion_v21/train/000000314914.png	10
./dataset/T2I/stable_diffusion_v21/train/000000225757.png	10
./dataset/T2I/stable_diffusion_v21/train/000000015517.png	10
./dataset/T2I/stable_diffusion_v21/train/000000384850.png	10
./dataset/T2I/stable_diffusion_v21/train/000000095155.png	10
./dataset/T2I/stable_diffusion_v21/train/000000319184.png	10
./dataset/T2I/stable_diffusion_v21/train/000000185409.png	10
./dataset/T2I/stable_diffusion_v21/train/000000294350.png	10
./dataset/T2I/stable_diffusion_v21/train/000000253695.png	10
./dataset/T2I/stable_diffusion_v21/train/000000135890.png	10
./dataset/T2I/stable_diffusion_v21/train/000000124798.png	10
./dataset/T2I/stable_diffusion_v21/train/000000355905.png	10
./dataset/T2I/stable_diffusion_v21/train/000000325347.png	10
./dataset/T2I/stable_diffusion_v21/train/000000537270.png	10
./dataset/T2I/stable_diffusion_v21/train/000000187745.png	10
./dataset/T2I/stable_diffusion_v21/train/000000355257.png	10
./dataset/T2I/stable_diffusion_v21/train/000000042563.png	10
./dataset/T2I/stable_diffusion_v21/train/000000383676.png	10
./dataset/T2I/stable_diffusion_v21/train/000000506933.png	10
./dataset/T2I/stable_diffusion_v21/train/000000029984.png	10
./dataset/T2I/stable_diffusion_v21/train/000000452122.png	10
./dataset/T2I/stable_diffusion_v21/train/000000478474.png	10
./dataset/T2I/stable_diffusion_v21/train/000000269316.png	10
./dataset/T2I/stable_diffusion_v21/train/000000136355.png	10
./dataset/T2I/stable_diffusion_v21/train/000000250127.png	10
./dataset/T2I/stable_diffusion_v21/train/000000267191.png	10
./dataset/T2I/stable_diffusion_v21/train/000000161128.png	10
./dataset/T2I/stable_diffusion_v21/train/000000308753.png	10
./dataset/T2I/stable_diffusion_v21/train/000000185472.png	10
./dataset/T2I/stable_diffusion_v21/train/000000482436.png	10
./dataset/T2I/stable_diffusion_v21/train/000000109798.png	10
./dataset/T2I/stable_diffusion_v21/train/000000114884.png	10
./dataset/T2I/stable_diffusion_v21/train/000000244592.png	10
./dataset/T2I/stable_diffusion_v21/train/000000404568.png	10
./dataset/T2I/stable_diffusion_v21/train/000000468501.png	10
./dataset/T2I/stable_diffusion_v21/train/000000056344.png	10
./dataset/T2I/stable_diffusion_v21/train/000000481413.png	10
./dataset/T2I/stable_diffusion_v21/train/000000576031.png	10
./dataset/T2I/stable_diffusion_v21/train/000000162581.png	10
./dataset/T2I/stable_diffusion_v21/train/000000334417.png	10
./dataset/T2I/stable_diffusion_v21/train/000000529568.png	10
./dataset/T2I/stable_diffusion_v21/train/000000489764.png	10
./dataset/T2I/stable_diffusion_v21/train/000000064523.png	10
./dataset/T2I/stable_diffusion_v21/train/000000116362.png	10
./dataset/T2I/stable_diffusion_v21/train/000000410712.png	10
./dataset/T2I/stable_diffusion_v21/train/000000396580.png	10
./dataset/T2I/stable_diffusion_v21/train/000000172977.png	10
./dataset/T2I/stable_diffusion_v21/train/000000392228.png	10
./dataset/T2I/stable_diffusion_v21/train/000000183500.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262938.png	10
./dataset/T2I/stable_diffusion_v21/train/000000579818.png	10
./dataset/T2I/stable_diffusion_v21/train/000000301563.png	10
./dataset/T2I/stable_diffusion_v21/train/000000306136.png	10
./dataset/T2I/stable_diffusion_v21/train/000000331317.png	10
./dataset/T2I/stable_diffusion_v21/train/000000549136.png	10
./dataset/T2I/stable_diffusion_v21/train/000000280710.png	10
./dataset/T2I/stable_diffusion_v21/train/000000222317.png	10
./dataset/T2I/stable_diffusion_v21/train/000000378453.png	10
./dataset/T2I/stable_diffusion_v21/train/000000404678.png	10
./dataset/T2I/stable_diffusion_v21/train/000000046872.png	10
./dataset/T2I/stable_diffusion_v21/train/000000153510.png	10
./dataset/T2I/stable_diffusion_v21/train/000000187144.png	10
./dataset/T2I/stable_diffusion_v21/train/000000127270.png	10
./dataset/T2I/stable_diffusion_v21/train/000000090108.png	10
./dataset/T2I/stable_diffusion_v21/train/000000405306.png	10
./dataset/T2I/stable_diffusion_v21/train/000000227491.png	10
./dataset/T2I/stable_diffusion_v21/train/000000499181.png	10
./dataset/T2I/stable_diffusion_v21/train/000000174482.png	10
./dataset/T2I/stable_diffusion_v21/train/000000206027.png	10
./dataset/T2I/stable_diffusion_v21/train/000000470924.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262631.png	10
./dataset/T2I/stable_diffusion_v21/train/000000402774.png	10
./dataset/T2I/stable_diffusion_v21/train/000000166563.png	10
./dataset/T2I/stable_diffusion_v21/train/000000359855.png	10
./dataset/T2I/stable_diffusion_v21/train/000000109055.png	10
./dataset/T2I/stable_diffusion_v21/train/000000378116.png	10
./dataset/T2I/stable_diffusion_v21/train/000000012667.png	10
./dataset/T2I/stable_diffusion_v21/train/000000439180.png	10
./dataset/T2I/stable_diffusion_v21/train/000000221872.png	10
./dataset/T2I/stable_diffusion_v21/train/000000261535.png	10
./dataset/T2I/stable_diffusion_v21/train/000000117744.png	10
./dataset/T2I/stable_diffusion_v21/train/000000533493.png	10
./dataset/T2I/stable_diffusion_v21/train/000000216636.png	10
./dataset/T2I/stable_diffusion_v21/train/000000546011.png	10
./dataset/T2I/stable_diffusion_v21/train/000000416330.png	10
./dataset/T2I/stable_diffusion_v21/train/000000018737.png	10
./dataset/T2I/stable_diffusion_v21/train/000000178982.png	10
./dataset/T2I/stable_diffusion_v21/train/000000351589.png	10
./dataset/T2I/stable_diffusion_v21/train/000000480944.png	10
./dataset/T2I/stable_diffusion_v21/train/000000497344.png	10
./dataset/T2I/stable_diffusion_v21/train/000000079031.png	10
./dataset/T2I/stable_diffusion_v21/train/000000027696.png	10
./dataset/T2I/stable_diffusion_v21/train/000000333772.png	10
./dataset/T2I/stable_diffusion_v21/train/000000458109.png	10
./dataset/T2I/stable_diffusion_v21/train/000000163290.png	10
./dataset/T2I/stable_diffusion_v21/train/000000025603.png	10
./dataset/T2I/stable_diffusion_v21/train/000000018833.png	10
./dataset/T2I/stable_diffusion_v21/train/000000267351.png	10
./dataset/T2I/stable_diffusion_v21/train/000000542127.png	10
./dataset/T2I/stable_diffusion_v21/train/000000379842.png	10
./dataset/T2I/stable_diffusion_v21/train/000000268831.png	10
./dataset/T2I/stable_diffusion_v21/train/000000119452.png	10
./dataset/T2I/stable_diffusion_v21/train/000000476787.png	10
./dataset/T2I/stable_diffusion_v21/train/000000154358.png	10
./dataset/T2I/stable_diffusion_v21/train/000000349837.png	10
./dataset/T2I/stable_diffusion_v21/train/000000565877.png	10
./dataset/T2I/stable_diffusion_v21/train/000000366141.png	10
./dataset/T2I/stable_diffusion_v21/train/000000140420.png	10
./dataset/T2I/stable_diffusion_v21/train/000000371677.png	10
./dataset/T2I/stable_diffusion_v21/train/000000297396.png	10
./dataset/T2I/stable_diffusion_v21/train/000000443498.png	10
./dataset/T2I/stable_diffusion_v21/train/000000224222.png	10
./dataset/T2I/stable_diffusion_v21/train/000000244496.png	10
./dataset/T2I/stable_diffusion_v21/train/000000017714.png	10
./dataset/T2I/stable_diffusion_v21/train/000000563603.png	10
./dataset/T2I/stable_diffusion_v21/train/000000421923.png	10
./dataset/T2I/stable_diffusion_v21/train/000000232646.png	10
./dataset/T2I/stable_diffusion_v21/train/000000550691.png	10
./dataset/T2I/stable_diffusion_v21/train/000000410510.png	10
./dataset/T2I/stable_diffusion_v21/train/000000170955.png	10
./dataset/T2I/stable_diffusion_v21/train/000000482978.png	10
./dataset/T2I/stable_diffusion_v21/train/000000505573.png	10
./dataset/T2I/stable_diffusion_v21/train/000000350002.png	10
./dataset/T2I/stable_diffusion_v21/train/000000469174.png	10
./dataset/T2I/stable_diffusion_v21/train/000000520301.png	10
./dataset/T2I/stable_diffusion_v21/train/000000485480.png	10
./dataset/T2I/stable_diffusion_v21/train/000000261732.png	10
./dataset/T2I/stable_diffusion_v21/train/000000339442.png	10
./dataset/T2I/stable_diffusion_v21/train/000000205834.png	10
./dataset/T2I/stable_diffusion_v21/train/000000574425.png	10
./dataset/T2I/stable_diffusion_v21/train/000000434479.png	10
./dataset/T2I/stable_diffusion_v21/train/000000474095.png	10
./dataset/T2I/stable_diffusion_v21/train/000000356125.png	10
./dataset/T2I/stable_diffusion_v21/train/000000555412.png	10
./dataset/T2I/stable_diffusion_v21/train/000000377393.png	10
./dataset/T2I/stable_diffusion_v21/train/000000128598.png	10
./dataset/T2I/stable_diffusion_v21/train/000000531134.png	10
./dataset/T2I/stable_diffusion_v21/train/000000199055.png	10
./dataset/T2I/stable_diffusion_v21/train/000000156292.png	10
./dataset/T2I/stable_diffusion_v21/train/000000070229.png	10
./dataset/T2I/stable_diffusion_v21/train/000000371042.png	10
./dataset/T2I/stable_diffusion_v21/train/000000250901.png	10
./dataset/T2I/stable_diffusion_v21/train/000000018150.png	10
./dataset/T2I/stable_diffusion_v21/train/000000230983.png	10
./dataset/T2I/stable_diffusion_v21/train/000000167898.png	10
./dataset/T2I/stable_diffusion_v21/train/000000410496.png	10
./dataset/T2I/stable_diffusion_v21/train/000000154705.png	10
./dataset/T2I/stable_diffusion_v21/train/000000001353.png	10
./dataset/T2I/stable_diffusion_v21/train/000000176232.png	10
./dataset/T2I/stable_diffusion_v21/train/000000491130.png	10
./dataset/T2I/stable_diffusion_v21/train/000000140270.png	10
./dataset/T2I/stable_diffusion_v21/train/000000026204.png	10
./dataset/T2I/stable_diffusion_v21/train/000000511999.png	10
./dataset/T2I/stable_diffusion_v21/train/000000452084.png	10
./dataset/T2I/stable_diffusion_v21/train/000000493799.png	10
./dataset/T2I/stable_diffusion_v21/train/000000532690.png	10
./dataset/T2I/stable_diffusion_v21/train/000000179487.png	10
./dataset/T2I/stable_diffusion_v21/train/000000253835.png	10
./dataset/T2I/stable_diffusion_v21/train/000000293858.png	10
./dataset/T2I/stable_diffusion_v21/train/000000066635.png	10
./dataset/T2I/stable_diffusion_v21/train/000000096549.png	10
./dataset/T2I/stable_diffusion_v21/train/000000084431.png	10
./dataset/T2I/stable_diffusion_v21/train/000000407943.png	10
./dataset/T2I/stable_diffusion_v21/train/000000269196.png	10
./dataset/T2I/stable_diffusion_v21/train/000000055072.png	10
./dataset/T2I/stable_diffusion_v21/train/000000311392.png	10
./dataset/T2I/stable_diffusion_v21/train/000000425226.png	10
./dataset/T2I/stable_diffusion_v21/train/000000079565.png	10
./dataset/T2I/stable_diffusion_v21/train/000000513567.png	10
./dataset/T2I/stable_diffusion_v21/train/000000281687.png	10
./dataset/T2I/stable_diffusion_v21/train/000000230362.png	10
./dataset/T2I/stable_diffusion_v21/train/000000442993.png	10
./dataset/T2I/stable_diffusion_v21/train/000000137727.png	10
./dataset/T2I/stable_diffusion_v21/train/000000046048.png	10
./dataset/T2I/stable_diffusion_v21/train/000000057238.png	10
./dataset/T2I/stable_diffusion_v21/train/000000521956.png	10
./dataset/T2I/stable_diffusion_v21/train/000000548555.png	10
./dataset/T2I/stable_diffusion_v21/train/000000042276.png	10
./dataset/T2I/stable_diffusion_v21/train/000000427655.png	10
./dataset/T2I/stable_diffusion_v21/train/000000414676.png	10
./dataset/T2I/stable_diffusion_v21/train/000000353051.png	10
./dataset/T2I/stable_diffusion_v21/train/000000334767.png	10
./dataset/T2I/stable_diffusion_v21/train/000000526392.png	10
./dataset/T2I/stable_diffusion_v21/train/000000554328.png	10
./dataset/T2I/stable_diffusion_v21/train/000000183104.png	10
./dataset/T2I/stable_diffusion_v21/train/000000218439.png	10
./dataset/T2I/stable_diffusion_v21/train/000000505565.png	10
./dataset/T2I/stable_diffusion_v21/train/000000024610.png	10
./dataset/T2I/stable_diffusion_v21/train/000000569825.png	10
./dataset/T2I/stable_diffusion_v21/train/000000343937.png	10
./dataset/T2I/stable_diffusion_v21/train/000000423229.png	10
./dataset/T2I/stable_diffusion_v21/train/000000137294.png	10
./dataset/T2I/stable_diffusion_v21/train/000000076416.png	10
./dataset/T2I/stable_diffusion_v21/train/000000062353.png	10
./dataset/T2I/stable_diffusion_v21/train/000000534639.png	10
./dataset/T2I/stable_diffusion_v21/train/000000558213.png	10
./dataset/T2I/stable_diffusion_v21/train/000000015079.png	10
./dataset/T2I/stable_diffusion_v21/train/000000543047.png	10
./dataset/T2I/stable_diffusion_v21/train/000000579900.png	10
./dataset/T2I/stable_diffusion_v21/train/000000378139.png	10
./dataset/T2I/stable_diffusion_v21/train/000000163057.png	10
./dataset/T2I/stable_diffusion_v21/train/000000335328.png	10
./dataset/T2I/stable_diffusion_v21/train/000000255747.png	10
./dataset/T2I/stable_diffusion_v21/train/000000417085.png	10
./dataset/T2I/stable_diffusion_v21/train/000000298251.png	10
./dataset/T2I/stable_diffusion_v21/train/000000068933.png	10
./dataset/T2I/stable_diffusion_v21/train/000000266082.png	10
./dataset/T2I/stable_diffusion_v21/train/000000388056.png	10
./dataset/T2I/stable_diffusion_v21/train/000000377946.png	10
./dataset/T2I/stable_diffusion_v21/train/000000228981.png	10
./dataset/T2I/stable_diffusion_v21/train/000000264441.png	10
./dataset/T2I/stable_diffusion_v21/train/000000383842.png	10
./dataset/T2I/stable_diffusion_v21/train/000000368900.png	10
./dataset/T2I/stable_diffusion_v21/train/000000214703.png	10
./dataset/T2I/stable_diffusion_v21/train/000000295809.png	10
./dataset/T2I/stable_diffusion_v21/train/000000179642.png	10
./dataset/T2I/stable_diffusion_v21/train/000000142585.png	10
./dataset/T2I/stable_diffusion_v21/train/000000166478.png	10
./dataset/T2I/stable_diffusion_v21/train/000000414261.png	10
./dataset/T2I/stable_diffusion_v21/train/000000497568.png	10
./dataset/T2I/stable_diffusion_v21/train/000000350488.png	10
./dataset/T2I/stable_diffusion_v21/train/000000546325.png	10
./dataset/T2I/stable_diffusion_v21/train/000000517523.png	10
./dataset/T2I/stable_diffusion_v21/train/000000434459.png	10
./dataset/T2I/stable_diffusion_v21/train/000000370270.png	10
./dataset/T2I/stable_diffusion_v21/train/000000009769.png	10
./dataset/T2I/stable_diffusion_v21/train/000000147725.png	10
./dataset/T2I/stable_diffusion_v21/train/000000198960.png	10
./dataset/T2I/stable_diffusion_v21/train/000000157213.png	10
./dataset/T2I/stable_diffusion_v21/train/000000455448.png	10
./dataset/T2I/stable_diffusion_v21/train/000000473118.png	10
./dataset/T2I/stable_diffusion_v21/train/000000248616.png	10
./dataset/T2I/stable_diffusion_v21/train/000000166521.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262048.png	10
./dataset/T2I/stable_diffusion_v21/train/000000359677.png	10
./dataset/T2I/stable_diffusion_v21/train/000000286708.png	10
./dataset/T2I/stable_diffusion_v21/train/000000166747.png	10
./dataset/T2I/stable_diffusion_v21/train/000000368294.png	10
./dataset/T2I/stable_diffusion_v21/train/000000521141.png	10
./dataset/T2I/stable_diffusion_v21/train/000000422886.png	10
./dataset/T2I/stable_diffusion_v21/train/000000377588.png	10
./dataset/T2I/stable_diffusion_v21/train/000000347254.png	10
./dataset/T2I/stable_diffusion_v21/train/000000416837.png	10
./dataset/T2I/stable_diffusion_v21/train/000000374083.png	10
./dataset/T2I/stable_diffusion_v21/train/000000427649.png	10
./dataset/T2I/stable_diffusion_v21/train/000000152686.png	10
./dataset/T2I/stable_diffusion_v21/train/000000188296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000266768.png	10
./dataset/T2I/stable_diffusion_v21/train/000000143068.png	10
./dataset/T2I/stable_diffusion_v21/train/000000263679.png	10
./dataset/T2I/stable_diffusion_v21/train/000000561256.png	10
./dataset/T2I/stable_diffusion_v21/train/000000283785.png	10
./dataset/T2I/stable_diffusion_v21/train/000000507235.png	10
./dataset/T2I/stable_diffusion_v21/train/000000201072.png	10
./dataset/T2I/stable_diffusion_v21/train/000000182805.png	10
./dataset/T2I/stable_diffusion_v21/train/000000357238.png	10
./dataset/T2I/stable_diffusion_v21/train/000000490125.png	10
./dataset/T2I/stable_diffusion_v21/train/000000263474.png	10
./dataset/T2I/stable_diffusion_v21/train/000000480936.png	10
./dataset/T2I/stable_diffusion_v21/train/000000391722.png	10
./dataset/T2I/stable_diffusion_v21/train/000000013348.png	10
./dataset/T2I/stable_diffusion_v21/train/000000441468.png	10
./dataset/T2I/stable_diffusion_v21/train/000000348708.png	10
./dataset/T2I/stable_diffusion_v21/train/000000463842.png	10
./dataset/T2I/stable_diffusion_v21/train/000000181796.png	10
./dataset/T2I/stable_diffusion_v21/train/000000058350.png	10
./dataset/T2I/stable_diffusion_v21/train/000000065288.png	10
./dataset/T2I/stable_diffusion_v21/train/000000491867.png	10
./dataset/T2I/stable_diffusion_v21/train/000000487583.png	10
./dataset/T2I/stable_diffusion_v21/train/000000553094.png	10
./dataset/T2I/stable_diffusion_v21/train/000000571313.png	10
./dataset/T2I/stable_diffusion_v21/train/000000111036.png	10
./dataset/T2I/stable_diffusion_v21/train/000000329080.png	10
./dataset/T2I/stable_diffusion_v21/train/000000152465.png	10
./dataset/T2I/stable_diffusion_v21/train/000000521819.png	10
./dataset/T2I/stable_diffusion_v21/train/000000378284.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426268.png	10
./dataset/T2I/stable_diffusion_v21/train/000000158744.png	10
./dataset/T2I/stable_diffusion_v21/train/000000145597.png	10
./dataset/T2I/stable_diffusion_v21/train/000000202001.png	10
./dataset/T2I/stable_diffusion_v21/train/000000420281.png	10
./dataset/T2I/stable_diffusion_v21/train/000000193429.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262227.png	10
./dataset/T2I/stable_diffusion_v21/train/000000074457.png	10
./dataset/T2I/stable_diffusion_v21/train/000000382030.png	10
./dataset/T2I/stable_diffusion_v21/train/000000286458.png	10
./dataset/T2I/stable_diffusion_v21/train/000000338532.png	10
./dataset/T2I/stable_diffusion_v21/train/000000522156.png	10
./dataset/T2I/stable_diffusion_v21/train/000000519338.png	10
./dataset/T2I/stable_diffusion_v21/train/000000094157.png	10
./dataset/T2I/stable_diffusion_v21/train/000000008021.png	10
./dataset/T2I/stable_diffusion_v21/train/000000248111.png	10
./dataset/T2I/stable_diffusion_v21/train/000000483531.png	10
./dataset/T2I/stable_diffusion_v21/train/000000523811.png	10
./dataset/T2I/stable_diffusion_v21/train/000000129054.png	10
./dataset/T2I/stable_diffusion_v21/train/000000451571.png	10
./dataset/T2I/stable_diffusion_v21/train/000000206135.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426376.png	10
./dataset/T2I/stable_diffusion_v21/train/000000207728.png	10
./dataset/T2I/stable_diffusion_v21/train/000000172935.png	10
./dataset/T2I/stable_diffusion_v21/train/000000537506.png	10
./dataset/T2I/stable_diffusion_v21/train/000000272416.png	10
./dataset/T2I/stable_diffusion_v21/train/000000567640.png	10
./dataset/T2I/stable_diffusion_v21/train/000000240767.png	10
./dataset/T2I/stable_diffusion_v21/train/000000564280.png	10
./dataset/T2I/stable_diffusion_v21/train/000000024567.png	10
./dataset/T2I/stable_diffusion_v21/train/000000402118.png	10
./dataset/T2I/stable_diffusion_v21/train/000000279774.png	10
./dataset/T2I/stable_diffusion_v21/train/000000048396.png	10
./dataset/T2I/stable_diffusion_v21/train/000000508639.png	10
./dataset/T2I/stable_diffusion_v21/train/000000180101.png	10
./dataset/T2I/stable_diffusion_v21/train/000000470779.png	10
./dataset/T2I/stable_diffusion_v21/train/000000445248.png	10
./dataset/T2I/stable_diffusion_v21/train/000000455352.png	10
./dataset/T2I/stable_diffusion_v21/train/000000501005.png	10
./dataset/T2I/stable_diffusion_v21/train/000000042070.png	10
./dataset/T2I/stable_diffusion_v21/train/000000214720.png	10
./dataset/T2I/stable_diffusion_v21/train/000000184324.png	10
./dataset/T2I/stable_diffusion_v21/train/000000476215.png	10
./dataset/T2I/stable_diffusion_v21/train/000000363207.png	10
./dataset/T2I/stable_diffusion_v21/train/000000231508.png	10
./dataset/T2I/stable_diffusion_v21/train/000000130613.png	10
./dataset/T2I/stable_diffusion_v21/train/000000396903.png	10
./dataset/T2I/stable_diffusion_v21/train/000000089761.png	10
./dataset/T2I/stable_diffusion_v21/train/000000277020.png	10
./dataset/T2I/stable_diffusion_v21/train/000000089670.png	10
./dataset/T2I/stable_diffusion_v21/train/000000035682.png	10
./dataset/T2I/stable_diffusion_v21/train/000000399205.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262440.png	10
./dataset/T2I/stable_diffusion_v21/train/000000125778.png	10
./dataset/T2I/stable_diffusion_v21/train/000000438226.png	10
./dataset/T2I/stable_diffusion_v21/train/000000356387.png	10
./dataset/T2I/stable_diffusion_v21/train/000000162732.png	10
./dataset/T2I/stable_diffusion_v21/train/000000067616.png	10
./dataset/T2I/stable_diffusion_v21/train/000000340451.png	10
./dataset/T2I/stable_diffusion_v21/train/000000191614.png	10
./dataset/T2I/stable_diffusion_v21/train/000000434297.png	10
./dataset/T2I/stable_diffusion_v21/train/000000461405.png	10
./dataset/T2I/stable_diffusion_v21/train/000000148999.png	10
./dataset/T2I/stable_diffusion_v21/train/000000088848.png	10
./dataset/T2I/stable_diffusion_v21/train/000000073118.png	10
./dataset/T2I/stable_diffusion_v21/train/000000000885.png	10
./dataset/T2I/stable_diffusion_v21/train/000000365208.png	10
./dataset/T2I/stable_diffusion_v21/train/000000071938.png	10
./dataset/T2I/stable_diffusion_v21/train/000000112997.png	10
./dataset/T2I/stable_diffusion_v21/train/000000228214.png	10
./dataset/T2I/stable_diffusion_v21/train/000000029187.png	10
./dataset/T2I/stable_diffusion_v21/train/000000163611.png	10
./dataset/T2I/stable_diffusion_v21/train/000000413689.png	10
./dataset/T2I/stable_diffusion_v21/train/000000375430.png	10
./dataset/T2I/stable_diffusion_v21/train/000000325306.png	10
./dataset/T2I/stable_diffusion_v21/train/000000406129.png	10
./dataset/T2I/stable_diffusion_v21/train/000000138819.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213816.png	10
./dataset/T2I/stable_diffusion_v21/train/000000516804.png	10
./dataset/T2I/stable_diffusion_v21/train/000000078823.png	10
./dataset/T2I/stable_diffusion_v21/train/000000054605.png	10
./dataset/T2I/stable_diffusion_v21/train/000000486479.png	10
./dataset/T2I/stable_diffusion_v21/train/000000438862.png	10
./dataset/T2I/stable_diffusion_v21/train/000000187585.png	10
./dataset/T2I/stable_diffusion_v21/train/000000218091.png	10
./dataset/T2I/stable_diffusion_v21/train/000000457078.png	10
./dataset/T2I/stable_diffusion_v21/train/000000387387.png	10
./dataset/T2I/stable_diffusion_v21/train/000000484029.png	10
./dataset/T2I/stable_diffusion_v21/train/000000133233.png	10
./dataset/T2I/stable_diffusion_v21/train/000000327306.png	10
./dataset/T2I/stable_diffusion_v21/train/000000396729.png	10
./dataset/T2I/stable_diffusion_v21/train/000000563604.png	10
./dataset/T2I/stable_diffusion_v21/train/000000234757.png	10
./dataset/T2I/stable_diffusion_v21/train/000000297085.png	10
./dataset/T2I/stable_diffusion_v21/train/000000457559.png	10
./dataset/T2I/stable_diffusion_v21/train/000000239537.png	10
./dataset/T2I/stable_diffusion_v21/train/000000371749.png	10
./dataset/T2I/stable_diffusion_v21/train/000000527960.png	10
./dataset/T2I/stable_diffusion_v21/train/000000391375.png	10
./dataset/T2I/stable_diffusion_v21/train/000000345941.png	10
./dataset/T2I/stable_diffusion_v21/train/000000016502.png	10
./dataset/T2I/stable_diffusion_v21/train/000000026690.png	10
./dataset/T2I/stable_diffusion_v21/train/000000057725.png	10
./dataset/T2I/stable_diffusion_v21/train/000000292155.png	10
./dataset/T2I/stable_diffusion_v21/train/000000015440.png	10
./dataset/T2I/stable_diffusion_v21/train/000000176857.png	10
./dataset/T2I/stable_diffusion_v21/train/000000376442.png	10
./dataset/T2I/stable_diffusion_v21/train/000000196442.png	10
./dataset/T2I/stable_diffusion_v21/train/000000058655.png	10
./dataset/T2I/stable_diffusion_v21/train/000000007386.png	10
./dataset/T2I/stable_diffusion_v21/train/000000221693.png	10
./dataset/T2I/stable_diffusion_v21/train/000000465179.png	10
./dataset/T2I/stable_diffusion_v21/train/000000060090.png	10
./dataset/T2I/stable_diffusion_v21/train/000000335954.png	10
./dataset/T2I/stable_diffusion_v21/train/000000224675.png	10
./dataset/T2I/stable_diffusion_v21/train/000000304545.png	10
./dataset/T2I/stable_diffusion_v21/train/000000404128.png	10
./dataset/T2I/stable_diffusion_v21/train/000000004395.png	10
./dataset/T2I/stable_diffusion_v21/train/000000196759.png	10
./dataset/T2I/stable_diffusion_v21/train/000000210273.png	10
./dataset/T2I/stable_diffusion_v21/train/000000212072.png	10
./dataset/T2I/stable_diffusion_v21/train/000000184978.png	10
./dataset/T2I/stable_diffusion_v21/train/000000357978.png	10
./dataset/T2I/stable_diffusion_v21/train/000000189806.png	10
./dataset/T2I/stable_diffusion_v21/train/000000333745.png	10
./dataset/T2I/stable_diffusion_v21/train/000000395343.png	10
./dataset/T2I/stable_diffusion_v21/train/000000227044.png	10
./dataset/T2I/stable_diffusion_v21/train/000000325991.png	10
./dataset/T2I/stable_diffusion_v21/train/000000286660.png	10
./dataset/T2I/stable_diffusion_v21/train/000000343466.png	10
./dataset/T2I/stable_diffusion_v21/train/000000527029.png	10
./dataset/T2I/stable_diffusion_v21/train/000000312552.png	10
./dataset/T2I/stable_diffusion_v21/train/000000465129.png	10
./dataset/T2I/stable_diffusion_v21/train/000000290619.png	10
./dataset/T2I/stable_diffusion_v21/train/000000296634.png	10
./dataset/T2I/stable_diffusion_v21/train/000000273711.png	10
./dataset/T2I/stable_diffusion_v21/train/000000366884.png	10
./dataset/T2I/stable_diffusion_v21/train/000000232684.png	10
./dataset/T2I/stable_diffusion_v21/train/000000502136.png	10
./dataset/T2I/stable_diffusion_v21/train/000000439773.png	10
./dataset/T2I/stable_diffusion_v21/train/000000012670.png	10
./dataset/T2I/stable_diffusion_v21/train/000000280325.png	10
./dataset/T2I/stable_diffusion_v21/train/000000109992.png	10
./dataset/T2I/stable_diffusion_v21/train/000000044260.png	10
./dataset/T2I/stable_diffusion_v21/train/000000581615.png	10
./dataset/T2I/stable_diffusion_v21/train/000000578967.png	10
./dataset/T2I/stable_diffusion_v21/train/000000184321.png	10
./dataset/T2I/stable_diffusion_v21/train/000000200252.png	10
./dataset/T2I/stable_diffusion_v21/train/000000350607.png	10
./dataset/T2I/stable_diffusion_v21/train/000000468577.png	10
./dataset/T2I/stable_diffusion_v21/train/000000267946.png	10
./dataset/T2I/stable_diffusion_v21/train/000000220310.png	10
./dataset/T2I/stable_diffusion_v21/train/000000172547.png	10
./dataset/T2I/stable_diffusion_v21/train/000000041888.png	10
./dataset/T2I/stable_diffusion_v21/train/000000092053.png	10
./dataset/T2I/stable_diffusion_v21/train/000000002592.png	10
./dataset/T2I/stable_diffusion_v21/train/000000458992.png	10
./dataset/T2I/stable_diffusion_v21/train/000000513283.png	10
./dataset/T2I/stable_diffusion_v21/train/000000106881.png	10
./dataset/T2I/stable_diffusion_v21/train/000000455085.png	10
./dataset/T2I/stable_diffusion_v21/train/000000150649.png	10
./dataset/T2I/stable_diffusion_v21/train/000000124975.png	10
./dataset/T2I/stable_diffusion_v21/train/000000352618.png	10
./dataset/T2I/stable_diffusion_v21/train/000000492992.png	10
./dataset/T2I/stable_diffusion_v21/train/000000189698.png	10
./dataset/T2I/stable_diffusion_v21/train/000000234660.png	10
./dataset/T2I/stable_diffusion_v21/train/000000440617.png	10
./dataset/T2I/stable_diffusion_v21/train/000000052462.png	10
./dataset/T2I/stable_diffusion_v21/train/000000074256.png	10
./dataset/T2I/stable_diffusion_v21/train/000000052413.png	10
./dataset/T2I/stable_diffusion_v21/train/000000078170.png	10
./dataset/T2I/stable_diffusion_v21/train/000000484296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000052996.png	10
./dataset/T2I/stable_diffusion_v21/train/000000516038.png	10
./dataset/T2I/stable_diffusion_v21/train/000000413552.png	10
./dataset/T2I/stable_diffusion_v21/train/000000388846.png	10
./dataset/T2I/stable_diffusion_v21/train/000000500477.png	10
./dataset/T2I/stable_diffusion_v21/train/000000252507.png	10
./dataset/T2I/stable_diffusion_v21/train/000000290843.png	10
./dataset/T2I/stable_diffusion_v21/train/000000161008.png	10
./dataset/T2I/stable_diffusion_v21/train/000000429281.png	10
./dataset/T2I/stable_diffusion_v21/train/000000361506.png	10
./dataset/T2I/stable_diffusion_v21/train/000000419312.png	10
./dataset/T2I/stable_diffusion_v21/train/000000309655.png	10
./dataset/T2I/stable_diffusion_v21/train/000000361268.png	10
./dataset/T2I/stable_diffusion_v21/train/000000054164.png	10
./dataset/T2I/stable_diffusion_v21/train/000000496854.png	10
./dataset/T2I/stable_diffusion_v21/train/000000010092.png	10
./dataset/T2I/stable_diffusion_v21/train/000000264535.png	10
./dataset/T2I/stable_diffusion_v21/train/000000298994.png	10
./dataset/T2I/stable_diffusion_v21/train/000000398742.png	10
./dataset/T2I/stable_diffusion_v21/train/000000445834.png	10
./dataset/T2I/stable_diffusion_v21/train/000000455157.png	10
./dataset/T2I/stable_diffusion_v21/train/000000489091.png	10
./dataset/T2I/stable_diffusion_v21/train/000000282912.png	10
./dataset/T2I/stable_diffusion_v21/train/000000530466.png	10
./dataset/T2I/stable_diffusion_v21/train/000000055528.png	10
./dataset/T2I/stable_diffusion_v21/train/000000090003.png	10
./dataset/T2I/stable_diffusion_v21/train/000000364297.png	10
./dataset/T2I/stable_diffusion_v21/train/000000344816.png	10
./dataset/T2I/stable_diffusion_v21/train/000000176799.png	10
./dataset/T2I/stable_diffusion_v21/train/000000132622.png	10
./dataset/T2I/stable_diffusion_v21/train/000000289059.png	10
./dataset/T2I/stable_diffusion_v21/train/000000148739.png	10
./dataset/T2I/stable_diffusion_v21/train/000000331280.png	10
./dataset/T2I/stable_diffusion_v21/train/000000550322.png	10
./dataset/T2I/stable_diffusion_v21/train/000000123585.png	10
./dataset/T2I/stable_diffusion_v21/train/000000342397.png	10
./dataset/T2I/stable_diffusion_v21/train/000000222863.png	10
./dataset/T2I/stable_diffusion_v21/train/000000271997.png	10
./dataset/T2I/stable_diffusion_v21/train/000000567432.png	10
./dataset/T2I/stable_diffusion_v21/train/000000084650.png	10
./dataset/T2I/stable_diffusion_v21/train/000000509699.png	10
./dataset/T2I/stable_diffusion_v21/train/000000424975.png	10
./dataset/T2I/stable_diffusion_v21/train/000000281693.png	10
./dataset/T2I/stable_diffusion_v21/train/000000229111.png	10
./dataset/T2I/stable_diffusion_v21/train/000000120777.png	10
./dataset/T2I/stable_diffusion_v21/train/000000568147.png	10
./dataset/T2I/stable_diffusion_v21/train/000000388258.png	10
./dataset/T2I/stable_diffusion_v21/train/000000184338.png	10
./dataset/T2I/stable_diffusion_v21/train/000000128654.png	10
./dataset/T2I/stable_diffusion_v21/train/000000322829.png	10
./dataset/T2I/stable_diffusion_v21/train/000000115245.png	10
./dataset/T2I/stable_diffusion_v21/train/000000420916.png	10
./dataset/T2I/stable_diffusion_v21/train/000000305343.png	10
./dataset/T2I/stable_diffusion_v21/train/000000151629.png	10
./dataset/T2I/stable_diffusion_v21/train/000000515982.png	10
./dataset/T2I/stable_diffusion_v21/train/000000071756.png	10
./dataset/T2I/stable_diffusion_v21/train/000000323202.png	10
./dataset/T2I/stable_diffusion_v21/train/000000507015.png	10
./dataset/T2I/stable_diffusion_v21/train/000000333069.png	10
./dataset/T2I/stable_diffusion_v21/train/000000442161.png	10
./dataset/T2I/stable_diffusion_v21/train/000000054592.png	10
./dataset/T2I/stable_diffusion_v21/train/000000143572.png	10
./dataset/T2I/stable_diffusion_v21/train/000000448448.png	10
./dataset/T2I/stable_diffusion_v21/train/000000031217.png	10
./dataset/T2I/stable_diffusion_v21/train/000000119677.png	10
./dataset/T2I/stable_diffusion_v21/train/000000570448.png	10
./dataset/T2I/stable_diffusion_v21/train/000000472298.png	10
./dataset/T2I/stable_diffusion_v21/train/000000322944.png	10
./dataset/T2I/stable_diffusion_v21/train/000000411817.png	10
./dataset/T2I/stable_diffusion_v21/train/000000530099.png	10
./dataset/T2I/stable_diffusion_v21/train/000000187990.png	10
./dataset/T2I/stable_diffusion_v21/train/000000464089.png	10
./dataset/T2I/stable_diffusion_v21/train/000000226058.png	10
./dataset/T2I/stable_diffusion_v21/train/000000007511.png	10
./dataset/T2I/stable_diffusion_v21/train/000000517832.png	10
./dataset/T2I/stable_diffusion_v21/train/000000167353.png	10
./dataset/T2I/stable_diffusion_v21/train/000000427500.png	10
./dataset/T2I/stable_diffusion_v21/train/000000128148.png	10
./dataset/T2I/stable_diffusion_v21/train/000000507975.png	10
./dataset/T2I/stable_diffusion_v21/train/000000049060.png	10
./dataset/T2I/stable_diffusion_v21/train/000000056288.png	10
./dataset/T2I/stable_diffusion_v21/train/000000178618.png	10
./dataset/T2I/stable_diffusion_v21/train/000000034417.png	10
./dataset/T2I/stable_diffusion_v21/train/000000334977.png	10
./dataset/T2I/stable_diffusion_v21/train/000000132931.png	10
./dataset/T2I/stable_diffusion_v21/train/000000017115.png	10
./dataset/T2I/stable_diffusion_v21/train/000000016958.png	10
./dataset/T2I/stable_diffusion_v21/train/000000565962.png	10
./dataset/T2I/stable_diffusion_v21/train/000000458702.png	10
./dataset/T2I/stable_diffusion_v21/train/000000108244.png	10
./dataset/T2I/stable_diffusion_v21/train/000000524742.png	10
./dataset/T2I/stable_diffusion_v21/train/000000501023.png	10
./dataset/T2I/stable_diffusion_v21/train/000000309452.png	10
./dataset/T2I/stable_diffusion_v21/train/000000560279.png	10
./dataset/T2I/stable_diffusion_v21/train/000000398905.png	10
./dataset/T2I/stable_diffusion_v21/train/000000471893.png	10
./dataset/T2I/stable_diffusion_v21/train/000000494759.png	10
./dataset/T2I/stable_diffusion_v21/train/000000126110.png	10
./dataset/T2I/stable_diffusion_v21/train/000000119828.png	10
./dataset/T2I/stable_diffusion_v21/train/000000419601.png	10
./dataset/T2I/stable_diffusion_v21/train/000000483667.png	10
./dataset/T2I/stable_diffusion_v21/train/000000146457.png	10
./dataset/T2I/stable_diffusion_v21/train/000000533958.png	10
./dataset/T2I/stable_diffusion_v21/train/000000182162.png	10
./dataset/T2I/stable_diffusion_v21/train/000000141671.png	10
./dataset/T2I/stable_diffusion_v21/train/000000044279.png	10
./dataset/T2I/stable_diffusion_v21/train/000000540928.png	10
./dataset/T2I/stable_diffusion_v21/train/000000079588.png	10
./dataset/T2I/stable_diffusion_v21/train/000000025394.png	10
./dataset/T2I/stable_diffusion_v21/train/000000159112.png	10
./dataset/T2I/stable_diffusion_v21/train/000000025139.png	10
./dataset/T2I/stable_diffusion_v21/train/000000028285.png	10
./dataset/T2I/stable_diffusion_v21/train/000000485071.png	10
./dataset/T2I/stable_diffusion_v21/train/000000540280.png	10
./dataset/T2I/stable_diffusion_v21/train/000000355169.png	10
./dataset/T2I/stable_diffusion_v21/train/000000333402.png	10
./dataset/T2I/stable_diffusion_v21/train/000000410428.png	10
./dataset/T2I/stable_diffusion_v21/train/000000142971.png	10
./dataset/T2I/stable_diffusion_v21/train/000000461275.png	10
./dataset/T2I/stable_diffusion_v21/train/000000035770.png	10
./dataset/T2I/stable_diffusion_v21/train/000000192670.png	10
./dataset/T2I/stable_diffusion_v21/train/000000246308.png	10
./dataset/T2I/stable_diffusion_v21/train/000000022705.png	10
./dataset/T2I/stable_diffusion_v21/train/000000540932.png	10
./dataset/T2I/stable_diffusion_v21/train/000000401862.png	10
./dataset/T2I/stable_diffusion_v21/train/000000252294.png	10
./dataset/T2I/stable_diffusion_v21/train/000000123131.png	10
./dataset/T2I/stable_diffusion_v21/train/000000466256.png	10
./dataset/T2I/stable_diffusion_v21/train/000000539883.png	10
./dataset/T2I/stable_diffusion_v21/train/000000336053.png	10
./dataset/T2I/stable_diffusion_v21/train/000000082696.png	10
./dataset/T2I/stable_diffusion_v21/train/000000186624.png	10
./dataset/T2I/stable_diffusion_v21/train/000000239857.png	10
./dataset/T2I/stable_diffusion_v21/train/000000046804.png	10
./dataset/T2I/stable_diffusion_v21/train/000000497867.png	10
./dataset/T2I/stable_diffusion_v21/train/000000548339.png	10
./dataset/T2I/stable_diffusion_v21/train/000000474881.png	10
./dataset/T2I/stable_diffusion_v21/train/000000450686.png	10
./dataset/T2I/stable_diffusion_v21/train/000000292415.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213224.png	10
./dataset/T2I/stable_diffusion_v21/train/000000570471.png	10
./dataset/T2I/stable_diffusion_v21/train/000000215723.png	10
./dataset/T2I/stable_diffusion_v21/train/000000067406.png	10
./dataset/T2I/stable_diffusion_v21/train/000000093437.png	10
./dataset/T2I/stable_diffusion_v21/train/000000451308.png	10
./dataset/T2I/stable_diffusion_v21/train/000000233238.png	10
./dataset/T2I/stable_diffusion_v21/train/000000088269.png	10
./dataset/T2I/stable_diffusion_v21/train/000000226883.png	10
./dataset/T2I/stable_diffusion_v21/train/000000356248.png	10
./dataset/T2I/stable_diffusion_v21/train/000000060102.png	10
./dataset/T2I/stable_diffusion_v21/train/000000189213.png	10
./dataset/T2I/stable_diffusion_v21/train/000000118515.png	10
./dataset/T2I/stable_diffusion_v21/train/000000142472.png	10
./dataset/T2I/stable_diffusion_v21/train/000000160666.png	10
./dataset/T2I/stable_diffusion_v21/train/000000312278.png	10
./dataset/T2I/stable_diffusion_v21/train/000000176606.png	10
./dataset/T2I/stable_diffusion_v21/train/000000226417.png	10
./dataset/T2I/stable_diffusion_v21/train/000000491757.png	10
./dataset/T2I/stable_diffusion_v21/train/000000163155.png	10
./dataset/T2I/stable_diffusion_v21/train/000000434996.png	10
./dataset/T2I/stable_diffusion_v21/train/000000037670.png	10
./dataset/T2I/stable_diffusion_v21/train/000000415716.png	10
./dataset/T2I/stable_diffusion_v21/train/000000228144.png	10
./dataset/T2I/stable_diffusion_v21/train/000000480212.png	10
./dataset/T2I/stable_diffusion_v21/train/000000082180.png	10
./dataset/T2I/stable_diffusion_v21/train/000000222299.png	10
./dataset/T2I/stable_diffusion_v21/train/000000039670.png	10
./dataset/T2I/stable_diffusion_v21/train/000000175443.png	10
./dataset/T2I/stable_diffusion_v21/train/000000067896.png	10
./dataset/T2I/stable_diffusion_v21/train/000000532058.png	10
./dataset/T2I/stable_diffusion_v21/train/000000297698.png	10
./dataset/T2I/stable_diffusion_v21/train/000000189828.png	10
./dataset/T2I/stable_diffusion_v21/train/000000291551.png	10
./dataset/T2I/stable_diffusion_v21/train/000000367095.png	10
./dataset/T2I/stable_diffusion_v21/train/000000112634.png	10
./dataset/T2I/stable_diffusion_v21/train/000000183127.png	10
./dataset/T2I/stable_diffusion_v21/train/000000161925.png	10
./dataset/T2I/stable_diffusion_v21/train/000000559547.png	10
./dataset/T2I/stable_diffusion_v21/train/000000077595.png	10
./dataset/T2I/stable_diffusion_v21/train/000000308328.png	10
./dataset/T2I/stable_diffusion_v21/train/000000546219.png	10
./dataset/T2I/stable_diffusion_v21/train/000000533816.png	10
./dataset/T2I/stable_diffusion_v21/train/000000177213.png	10
./dataset/T2I/stable_diffusion_v21/train/000000505451.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213593.png	10
./dataset/T2I/stable_diffusion_v21/train/000000135673.png	10
./dataset/T2I/stable_diffusion_v21/train/000000290768.png	10
./dataset/T2I/stable_diffusion_v21/train/000000293390.png	10
./dataset/T2I/stable_diffusion_v21/train/000000054593.png	10
./dataset/T2I/stable_diffusion_v21/train/000000254516.png	10
./dataset/T2I/stable_diffusion_v21/train/000000377497.png	10
./dataset/T2I/stable_diffusion_v21/train/000000236784.png	10
./dataset/T2I/stable_diffusion_v21/train/000000565989.png	10
./dataset/T2I/stable_diffusion_v21/train/000000512929.png	10
./dataset/T2I/stable_diffusion_v21/train/000000322724.png	10
./dataset/T2I/stable_diffusion_v21/train/000000377000.png	10
./dataset/T2I/stable_diffusion_v21/train/000000521719.png	10
./dataset/T2I/stable_diffusion_v21/train/000000576955.png	10
./dataset/T2I/stable_diffusion_v21/train/000000131138.png	10
./dataset/T2I/stable_diffusion_v21/train/000000502599.png	10
./dataset/T2I/stable_diffusion_v21/train/000000186873.png	10
./dataset/T2I/stable_diffusion_v21/train/000000473974.png	10
./dataset/T2I/stable_diffusion_v21/train/000000476770.png	10
./dataset/T2I/stable_diffusion_v21/train/000000130826.png	10
./dataset/T2I/stable_diffusion_v21/train/000000195918.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213830.png	10
./dataset/T2I/stable_diffusion_v21/train/000000311180.png	10
./dataset/T2I/stable_diffusion_v21/train/000000475223.png	10
./dataset/T2I/stable_diffusion_v21/train/000000523957.png	10
./dataset/T2I/stable_diffusion_v21/train/000000399560.png	10
./dataset/T2I/stable_diffusion_v21/train/000000438955.png	10
./dataset/T2I/stable_diffusion_v21/train/000000481386.png	10
./dataset/T2I/stable_diffusion_v21/train/000000269682.png	10
./dataset/T2I/stable_diffusion_v21/train/000000194746.png	10
./dataset/T2I/stable_diffusion_v21/train/000000399655.png	10
./dataset/T2I/stable_diffusion_v21/train/000000229948.png	10
./dataset/T2I/stable_diffusion_v21/train/000000407298.png	10
./dataset/T2I/stable_diffusion_v21/train/000000129322.png	10
./dataset/T2I/stable_diffusion_v21/train/000000032817.png	10
./dataset/T2I/stable_diffusion_v21/train/000000020059.png	10
./dataset/T2I/stable_diffusion_v21/train/000000319607.png	10
./dataset/T2I/stable_diffusion_v21/train/000000442009.png	10
./dataset/T2I/stable_diffusion_v21/train/000000121031.png	10
./dataset/T2I/stable_diffusion_v21/train/000000127987.png	10
./dataset/T2I/stable_diffusion_v21/train/000000293804.png	10
./dataset/T2I/stable_diffusion_v21/train/000000326128.png	10
./dataset/T2I/stable_diffusion_v21/train/000000445999.png	10
./dataset/T2I/stable_diffusion_v21/train/000000269942.png	10
./dataset/T2I/stable_diffusion_v21/train/000000056127.png	10
./dataset/T2I/stable_diffusion_v21/train/000000165681.png	10
./dataset/T2I/stable_diffusion_v21/train/000000195165.png	10
./dataset/T2I/stable_diffusion_v21/train/000000090062.png	10
./dataset/T2I/stable_diffusion_v21/train/000000246522.png	10
./dataset/T2I/stable_diffusion_v21/train/000000233727.png	10
./dataset/T2I/stable_diffusion_v21/train/000000468632.png	10
./dataset/T2I/stable_diffusion_v21/train/000000492758.png	10
./dataset/T2I/stable_diffusion_v21/train/000000577959.png	10
./dataset/T2I/stable_diffusion_v21/train/000000368456.png	10
./dataset/T2I/stable_diffusion_v21/train/000000072281.png	10
./dataset/T2I/stable_diffusion_v21/train/000000497628.png	10
./dataset/T2I/stable_diffusion_v21/train/000000329447.png	10
./dataset/T2I/stable_diffusion_v21/train/000000323151.png	10
./dataset/T2I/stable_diffusion_v21/train/000000279927.png	10
./dataset/T2I/stable_diffusion_v21/train/000000107339.png	10
./dataset/T2I/stable_diffusion_v21/train/000000499266.png	10
./dataset/T2I/stable_diffusion_v21/train/000000007088.png	10
./dataset/T2I/stable_diffusion_v21/train/000000189226.png	10
./dataset/T2I/stable_diffusion_v21/train/000000262487.png	10
./dataset/T2I/stable_diffusion_v21/train/000000135561.png	10
./dataset/T2I/stable_diffusion_v21/train/000000357941.png	10
./dataset/T2I/stable_diffusion_v21/train/000000154718.png	10
./dataset/T2I/stable_diffusion_v21/train/000000564127.png	10
./dataset/T2I/stable_diffusion_v21/train/000000512248.png	10
./dataset/T2I/stable_diffusion_v21/train/000000239274.png	10
./dataset/T2I/stable_diffusion_v21/train/000000368335.png	10
./dataset/T2I/stable_diffusion_v21/train/000000414133.png	10
./dataset/T2I/stable_diffusion_v21/train/000000395701.png	10
./dataset/T2I/stable_diffusion_v21/train/000000160864.png	10
./dataset/T2I/stable_diffusion_v21/train/000000251065.png	10
./dataset/T2I/stable_diffusion_v21/train/000000078032.png	10
./dataset/T2I/stable_diffusion_v21/train/000000470773.png	10
./dataset/T2I/stable_diffusion_v21/train/000000169169.png	10
./dataset/T2I/stable_diffusion_v21/train/000000577584.png	10
./dataset/T2I/stable_diffusion_v21/train/000000146489.png	10
./dataset/T2I/stable_diffusion_v21/train/000000035279.png	10
./dataset/T2I/stable_diffusion_v21/train/000000510095.png	10
./dataset/T2I/stable_diffusion_v21/train/000000282046.png	10
./dataset/T2I/stable_diffusion_v21/train/000000430973.png	10
./dataset/T2I/stable_diffusion_v21/train/000000370677.png	10
./dataset/T2I/stable_diffusion_v21/train/000000400922.png	10
./dataset/T2I/stable_diffusion_v21/train/000000183709.png	10
./dataset/T2I/stable_diffusion_v21/train/000000079229.png	10
./dataset/T2I/stable_diffusion_v21/train/000000015956.png	10
./dataset/T2I/stable_diffusion_v21/train/000000441543.png	10
./dataset/T2I/stable_diffusion_v21/train/000000192191.png	10
./dataset/T2I/stable_diffusion_v21/train/000000128476.png	10
./dataset/T2I/stable_diffusion_v21/train/000000154947.png	10
./dataset/T2I/stable_diffusion_v21/train/000000514376.png	10
./dataset/T2I/stable_diffusion_v21/train/000000509656.png	10
./dataset/T2I/stable_diffusion_v21/train/000000437898.png	10
./dataset/T2I/stable_diffusion_v21/train/000000558558.png	10
./dataset/T2I/stable_diffusion_v21/train/000000018519.png	10
./dataset/T2I/stable_diffusion_v21/train/000000473219.png	10
./dataset/T2I/stable_diffusion_v21/train/000000424135.png	10
./dataset/T2I/stable_diffusion_v21/train/000000463174.png	10
./dataset/T2I/stable_diffusion_v21/train/000000418062.png	10
./dataset/T2I/stable_diffusion_v21/train/000000149568.png	10
./dataset/T2I/stable_diffusion_v21/train/000000153343.png	10
./dataset/T2I/stable_diffusion_v21/train/000000023359.png	10
./dataset/T2I/stable_diffusion_v21/train/000000078404.png	10
./dataset/T2I/stable_diffusion_v21/train/000000572900.png	10
./dataset/T2I/stable_diffusion_v21/train/000000235241.png	10
./dataset/T2I/stable_diffusion_v21/train/000000010363.png	10
./dataset/T2I/stable_diffusion_v21/train/000000153782.png	10
./dataset/T2I/stable_diffusion_v21/train/000000348012.png	10
./dataset/T2I/stable_diffusion_v21/train/000000065074.png	10
./dataset/T2I/stable_diffusion_v21/train/000000289659.png	10
./dataset/T2I/stable_diffusion_v21/train/000000450075.png	10
./dataset/T2I/stable_diffusion_v21/train/000000464689.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213171.png	10
./dataset/T2I/stable_diffusion_v21/train/000000278353.png	10
./dataset/T2I/stable_diffusion_v21/train/000000348481.png	10
./dataset/T2I/stable_diffusion_v21/train/000000570688.png	10
./dataset/T2I/stable_diffusion_v21/train/000000137106.png	10
./dataset/T2I/stable_diffusion_v21/train/000000084477.png	10
./dataset/T2I/stable_diffusion_v21/train/000000474786.png	10
./dataset/T2I/stable_diffusion_v21/train/000000349678.png	10
./dataset/T2I/stable_diffusion_v21/train/000000401250.png	10
./dataset/T2I/stable_diffusion_v21/train/000000463690.png	10
./dataset/T2I/stable_diffusion_v21/train/000000460494.png	10
./dataset/T2I/stable_diffusion_v21/train/000000301867.png	10
./dataset/T2I/stable_diffusion_v21/train/000000445792.png	10
./dataset/T2I/stable_diffusion_v21/train/000000363840.png	10
./dataset/T2I/stable_diffusion_v21/train/000000289702.png	10
./dataset/T2I/stable_diffusion_v21/train/000000071226.png	10
./dataset/T2I/stable_diffusion_v21/train/000000356169.png	10
./dataset/T2I/stable_diffusion_v21/train/000000402334.png	10
./dataset/T2I/stable_diffusion_v21/train/000000274460.png	10
./dataset/T2I/stable_diffusion_v21/train/000000505789.png	10
./dataset/T2I/stable_diffusion_v21/train/000000199236.png	10
./dataset/T2I/stable_diffusion_v21/train/000000562561.png	10
./dataset/T2I/stable_diffusion_v21/train/000000177934.png	10
./dataset/T2I/stable_diffusion_v21/train/000000324927.png	10
./dataset/T2I/stable_diffusion_v21/train/000000109441.png	10
./dataset/T2I/stable_diffusion_v21/train/000000067534.png	10
./dataset/T2I/stable_diffusion_v21/train/000000474344.png	10
./dataset/T2I/stable_diffusion_v21/train/000000086956.png	10
./dataset/T2I/stable_diffusion_v21/train/000000378515.png	10
./dataset/T2I/stable_diffusion_v21/train/000000131556.png	10
./dataset/T2I/stable_diffusion_v21/train/000000017029.png	10
./dataset/T2I/stable_diffusion_v21/train/000000300155.png	10
./dataset/T2I/stable_diffusion_v21/train/000000414510.png	10
./dataset/T2I/stable_diffusion_v21/train/000000104666.png	10
./dataset/T2I/stable_diffusion_v21/train/000000511398.png	10
./dataset/T2I/stable_diffusion_v21/train/000000085911.png	10
./dataset/T2I/stable_diffusion_v21/train/000000328030.png	10
./dataset/T2I/stable_diffusion_v21/train/000000347930.png	10
./dataset/T2I/stable_diffusion_v21/train/000000167486.png	10
./dataset/T2I/stable_diffusion_v21/train/000000242946.png	10
./dataset/T2I/stable_diffusion_v21/train/000000216296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000549055.png	10
./dataset/T2I/stable_diffusion_v21/train/000000346707.png	10
./dataset/T2I/stable_diffusion_v21/train/000000423798.png	10
./dataset/T2I/stable_diffusion_v21/train/000000006894.png	10
./dataset/T2I/stable_diffusion_v21/train/000000504000.png	10
./dataset/T2I/stable_diffusion_v21/train/000000369323.png	10
./dataset/T2I/stable_diffusion_v21/train/000000242678.png	10
./dataset/T2I/stable_diffusion_v21/train/000000261796.png	10
./dataset/T2I/stable_diffusion_v21/train/000000011051.png	10
./dataset/T2I/stable_diffusion_v21/train/000000271471.png	10
./dataset/T2I/stable_diffusion_v21/train/000000212800.png	10
./dataset/T2I/stable_diffusion_v21/train/000000150930.png	10
./dataset/T2I/stable_diffusion_v21/train/000000433134.png	10
./dataset/T2I/stable_diffusion_v21/train/000000047740.png	10
./dataset/T2I/stable_diffusion_v21/train/000000344611.png	10
./dataset/T2I/stable_diffusion_v21/train/000000183246.png	10
./dataset/T2I/stable_diffusion_v21/train/000000199442.png	10
./dataset/T2I/stable_diffusion_v21/train/000000442456.png	10
./dataset/T2I/stable_diffusion_v21/train/000000001818.png	10
./dataset/T2I/stable_diffusion_v21/train/000000166664.png	10
./dataset/T2I/stable_diffusion_v21/train/000000090891.png	10
./dataset/T2I/stable_diffusion_v21/train/000000408696.png	10
./dataset/T2I/stable_diffusion_v21/train/000000229747.png	10
./dataset/T2I/stable_diffusion_v21/train/000000389812.png	10
./dataset/T2I/stable_diffusion_v21/train/000000252716.png	10
./dataset/T2I/stable_diffusion_v21/train/000000278006.png	10
./dataset/T2I/stable_diffusion_v21/train/000000459437.png	10
./dataset/T2I/stable_diffusion_v21/train/000000433192.png	10
./dataset/T2I/stable_diffusion_v21/train/000000066771.png	10
./dataset/T2I/stable_diffusion_v21/train/000000341921.png	10
./dataset/T2I/stable_diffusion_v21/train/000000110282.png	10
./dataset/T2I/stable_diffusion_v21/train/000000554002.png	10
./dataset/T2I/stable_diffusion_v21/train/000000534827.png	10
./dataset/T2I/stable_diffusion_v21/train/000000003553.png	10
./dataset/T2I/stable_diffusion_v21/train/000000428867.png	10
./dataset/T2I/stable_diffusion_v21/train/000000488270.png	10
./dataset/T2I/stable_diffusion_v21/train/000000213422.png	10
./dataset/T2I/stable_diffusion_v21/train/000000132329.png	10
./dataset/T2I/stable_diffusion_v21/train/000000443969.png	10
./dataset/T2I/stable_diffusion_v21/train/000000239318.png	10
./dataset/T2I/stable_diffusion_v21/train/000000549674.png	10
./dataset/T2I/stable_diffusion_v21/train/000000089296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000289741.png	10
./dataset/T2I/stable_diffusion_v21/train/000000257896.png	10
./dataset/T2I/stable_diffusion_v21/train/000000417249.png	10
./dataset/T2I/stable_diffusion_v21/train/000000128658.png	10
./dataset/T2I/stable_diffusion_v21/train/000000022623.png	10
./dataset/T2I/stable_diffusion_v21/train/000000458255.png	10
./dataset/T2I/stable_diffusion_v21/train/000000369751.png	10
./dataset/T2I/stable_diffusion_v21/train/000000051610.png	10
./dataset/T2I/stable_diffusion_v21/train/000000094614.png	10
./dataset/T2I/stable_diffusion_v21/train/000000466339.png	10
./dataset/T2I/stable_diffusion_v21/train/000000459396.png	10
./dataset/T2I/stable_diffusion_v21/train/000000242411.png	10
./dataset/T2I/stable_diffusion_v21/train/000000514914.png	10
./dataset/T2I/stable_diffusion_v21/train/000000249129.png	10
./dataset/T2I/stable_diffusion_v21/train/000000106266.png	10
./dataset/T2I/stable_diffusion_v21/train/000000402992.png	10
./dataset/T2I/stable_diffusion_v21/train/000000208901.png	10
./dataset/T2I/stable_diffusion_v21/train/000000350122.png	10
./dataset/T2I/stable_diffusion_v21/train/000000351096.png	10
./dataset/T2I/stable_diffusion_v21/train/000000069106.png	10
./dataset/T2I/stable_diffusion_v21/train/000000002923.png	10
./dataset/T2I/stable_diffusion_v21/train/000000206271.png	10
./dataset/T2I/stable_diffusion_v21/train/000000392481.png	10
./dataset/T2I/stable_diffusion_v21/train/000000284296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426372.png	10
./dataset/T2I/stable_diffusion_v21/train/000000031269.png	10
./dataset/T2I/stable_diffusion_v21/train/000000458410.png	10
./dataset/T2I/stable_diffusion_v21/train/000000411953.png	10
./dataset/T2I/stable_diffusion_v21/train/000000496571.png	10
./dataset/T2I/stable_diffusion_v21/train/000000493334.png	10
./dataset/T2I/stable_diffusion_v21/train/000000163258.png	10
./dataset/T2I/stable_diffusion_v21/train/000000022192.png	10
./dataset/T2I/stable_diffusion_v21/train/000000136033.png	10
./dataset/T2I/stable_diffusion_v21/train/000000418959.png	10
./dataset/T2I/stable_diffusion_v21/train/000000263969.png	10
./dataset/T2I/stable_diffusion_v21/train/000000248400.png	10
./dataset/T2I/stable_diffusion_v21/train/000000144984.png	10
./dataset/T2I/stable_diffusion_v21/train/000000258883.png	10
./dataset/T2I/stable_diffusion_v21/train/000000466416.png	10
./dataset/T2I/stable_diffusion_v21/train/000000199551.png	10
./dataset/T2I/stable_diffusion_v21/train/000000174018.png	10
./dataset/T2I/stable_diffusion_v21/train/000000318908.png	10
./dataset/T2I/stable_diffusion_v21/train/000000050145.png	10
./dataset/T2I/stable_diffusion_v21/train/000000493442.png	10
./dataset/T2I/stable_diffusion_v21/train/000000010707.png	10
./dataset/T2I/stable_diffusion_v21/train/000000067213.png	10
./dataset/T2I/stable_diffusion_v21/train/000000245764.png	10
./dataset/T2I/stable_diffusion_v21/train/000000019402.png	10
./dataset/T2I/stable_diffusion_v21/train/000000320642.png	10
./dataset/T2I/stable_diffusion_v21/train/000000159684.png	10
./dataset/T2I/stable_diffusion_v21/train/000000566042.png	10
./dataset/T2I/stable_diffusion_v21/train/000000216739.png	10
./dataset/T2I/stable_diffusion_v21/train/000000108503.png	10
./dataset/T2I/stable_diffusion_v21/train/000000003845.png	10
./dataset/T2I/stable_diffusion_v21/train/000000475484.png	10
./dataset/T2I/stable_diffusion_v21/train/000000425925.png	10
./dataset/T2I/stable_diffusion_v21/train/000000404601.png	10
./dataset/T2I/stable_diffusion_v21/train/000000530470.png	10
./dataset/T2I/stable_diffusion_v21/train/000000533145.png	10
./dataset/T2I/stable_diffusion_v21/train/000000471450.png	10
./dataset/T2I/stable_diffusion_v21/train/000000267169.png	10
./dataset/T2I/stable_diffusion_v21/train/000000555012.png	10
./dataset/T2I/stable_diffusion_v21/train/000000093717.png	10
./dataset/T2I/stable_diffusion_v21/train/000000291861.png	10
./dataset/T2I/stable_diffusion_v21/train/000000073326.png	10
./dataset/T2I/stable_diffusion_v21/train/000000397133.png	10
./dataset/T2I/stable_diffusion_v21/train/000000023034.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426297.png	10
./dataset/T2I/stable_diffusion_v21/train/000000325483.png	10
./dataset/T2I/stable_diffusion_v21/train/000000147729.png	10
./dataset/T2I/stable_diffusion_v21/train/000000301061.png	10
./dataset/T2I/stable_diffusion_v21/train/000000300276.png	10
./dataset/T2I/stable_diffusion_v21/train/000000443844.png	10
./dataset/T2I/stable_diffusion_v21/train/000000384136.png	10
./dataset/T2I/stable_diffusion_v21/train/000000398028.png	10
./dataset/T2I/stable_diffusion_v21/train/000000224119.png	10
./dataset/T2I/stable_diffusion_v21/train/000000343976.png	10
./dataset/T2I/stable_diffusion_v21/train/000000372317.png	10
./dataset/T2I/stable_diffusion_v21/train/000000110042.png	10
./dataset/T2I/stable_diffusion_v21/train/000000266892.png	10
./dataset/T2I/stable_diffusion_v21/train/000000343934.png	10
./dataset/T2I/stable_diffusion_v21/train/000000180296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000005060.png	10
./dataset/T2I/stable_diffusion_v21/train/000000439290.png	10
./dataset/T2I/stable_diffusion_v21/train/000000553776.png	10
./dataset/T2I/stable_diffusion_v21/train/000000494863.png	10
./dataset/T2I/stable_diffusion_v21/train/000000281179.png	10
./dataset/T2I/stable_diffusion_v21/train/000000167067.png	10
./dataset/T2I/stable_diffusion_v21/train/000000528524.png	10
./dataset/T2I/stable_diffusion_v21/train/000000194716.png	10
./dataset/T2I/stable_diffusion_v21/train/000000025057.png	10
./dataset/T2I/stable_diffusion_v21/train/000000350019.png	10
./dataset/T2I/stable_diffusion_v21/train/000000092091.png	10
./dataset/T2I/stable_diffusion_v21/train/000000133819.png	10
./dataset/T2I/stable_diffusion_v21/train/000000036539.png	10
./dataset/T2I/stable_diffusion_v21/train/000000426253.png	10
./dataset/T2I/stable_diffusion_v21/train/000000087144.png	10
./dataset/T2I/stable_diffusion_v21/train/000000240754.png	10
./dataset/T2I/stable_diffusion_v21/train/000000409867.png	10
./dataset/T2I/stable_diffusion_v21/train/000000351823.png	10
./dataset/T2I/stable_diffusion_v21/train/000000322959.png	10
./dataset/T2I/stable_diffusion_v21/train/000000580294.png	10
./dataset/T2I/stable_diffusion_v21/train/000000489924.png	10
./dataset/T2I/stable_diffusion_v21/train/000000022371.png	10
./dataset/T2I/stable_diffusion_v21/train/000000022755.png	10
./dataset/T2I/stable_diffusion_v21/train/000000135902.png	10
./dataset/T2I/stable_diffusion_v21/train/000000031093.png	10
./dataset/T2I/stable_diffusion_v21/train/000000384808.png	10
./dataset/T2I/stable_diffusion_v21/train/000000201426.png	10
./dataset/T2I/stable_diffusion_v21/train/000000457262.png	10
./dataset/T2I/stable_diffusion_v21/train/000000464358.png	10
./dataset/T2I/stable_diffusion_v21/train/000000366711.png	10
./dataset/T2I/stable_diffusion_v21/train/000000006460.png	10
./dataset/T2I/stable_diffusion_v21/train/000000178469.png	10
./dataset/T2I/stable_diffusion_v21/train/000000172571.png	10
./dataset/T2I/stable_diffusion_v21/train/000000232649.png	10
./dataset/T2I/stable_diffusion_v21/train/000000039484.png	10
./dataset/T2I/stable_diffusion_v21/train/000000528862.png	10
./dataset/T2I/stable_diffusion_v21/train/000000311190.png	10
./dataset/T2I/stable_diffusion_v21/train/000000109118.png	10
./dataset/T2I/stable_diffusion_v21/train/000000136715.png	10
./dataset/T2I/stable_diffusion_v21/train/000000106330.png	10
./dataset/T2I/stable_diffusion_v21/train/000000283717.png	10
./dataset/T2I/stable_diffusion_v21/train/000000491366.png	10
./dataset/T2I/stable_diffusion_v21/train/000000222735.png	10
./dataset/T2I/stable_diffusion_v21/train/000000256868.png	10
./dataset/T2I/stable_diffusion_v21/train/000000196009.png	10
./dataset/T2I/stable_diffusion_v21/train/000000509735.png	10
./dataset/T2I/stable_diffusion_v21/train/000000259854.png	10
./dataset/T2I/stable_diffusion_v21/train/000000535253.png	10
./dataset/T2I/stable_diffusion_v21/train/000000447313.png	10
./dataset/T2I/stable_diffusion_v21/train/000000449909.png	10
./dataset/T2I/stable_diffusion_v21/train/000000415536.png	10
./dataset/T2I/stable_diffusion_v21/train/000000155443.png	10
./dataset/T2I/stable_diffusion_v21/train/000000006471.png	10
./dataset/T2I/stable_diffusion_v21/train/000000042296.png	10
./dataset/T2I/stable_diffusion_v21/train/000000328238.png	10
./dataset/T2I/stable_diffusion_v21/train/000000347174.png	10
./dataset/T2I/stable_diffusion_v21/train/000000002473.png	10
./dataset/T2I/stable_diffusion_v21/train/000000360564.png	10
./dataset/T2I/stable_diffusion_v21/train/000000193348.png	10
./dataset/T2I/stable_diffusion_v21/train/000000068387.png	10
./dataset/T2I/stable_diffusion_v21/train/000000492968.png	10
'''

paths = parse_image_paths(paths)
for path in paths:
    main(path)
    